In [1]:
import warnings
warnings.filterwarnings('ignore')

import warnings
import pandas as pd
import geopandas as gpd
from pathlib import Path

# Suprimir warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None


def load_geodata(file_path):
    """Carrega dados geoespaciais de um arquivo."""
    try:
        return gpd.read_file(file_path)
    except Exception as e:
        raise FileNotFoundError(f"Erro ao ler arquivo {file_path}: {e}")


def get_user_input():
    """Solicita entrada do usuário para distrito e concelho."""
    distrito = input("Informe o nome do distrito (ex: Évora, Lisboa, Porto): ").strip()
    concelho = input("Informe o nome do concelho (ex: Évora, Sintra, Matosinhos): ").strip()
    return distrito, concelho


def filter_distrito(distritos_gdf, distrito_nome):
    """Filtra e remove o distrito selecionado dos dados."""
    distrito_normalizado = distrito_nome.lower()
    distrito_encontrado = distritos_gdf[
        distritos_gdf["name"].str.strip().str.lower() == distrito_normalizado
    ]

    if distrito_encontrado.empty:
        raise ValueError(f"Distrito '{distrito_nome}' não encontrado.")

    outros_distritos = distritos_gdf[
        distritos_gdf["name"].str.strip().str.lower() != distrito_normalizado
    ]
    outros_distritos["id"] = "PRO-" + outros_distritos["id"].astype(str)

    return outros_distritos


def process_concelhos(concelhos_gdf, concelho_nome, distrito_nome):
    """Processa concelhos do distrito selecionado, separando o concelho de interesse."""
    distrito_normalizado = distrito_nome.lower()
    concelho_normalizado = concelho_nome.lower()

    # Filtra apenas os concelhos do distrito informado
    concelhos_do_distrito = concelhos_gdf[
        concelhos_gdf["distrito"].str.strip().str.lower() == distrito_normalizado
    ]

    if concelhos_do_distrito.empty:
        raise ValueError(f"Nenhum concelho encontrado para o distrito '{distrito_nome}'.")

    # Verifica se o concelho existe no distrito
    concelho_selecionado = concelhos_do_distrito[
        concelhos_do_distrito["name"].str.strip().str.lower() == concelho_normalizado
    ]

    if concelho_selecionado.empty:
        raise ValueError(f"Concelho '{concelho_nome}' não encontrado no distrito '{distrito_nome}'.")

    # Configura o concelho selecionado
    concelho_selecionado = concelho_selecionado.copy()
    concelho_selecionado["id"] = "AOI-" + concelho_selecionado["id"].astype(str)
    concelho_selecionado["poly_type"] = "core"

    # Outros concelhos do mesmo distrito (exceto o selecionado)
    outros_concelhos_distrito = concelhos_do_distrito[
        concelhos_do_distrito["name"].str.strip().str.lower() != concelho_normalizado
    ]
    outros_concelhos_distrito["id"] = "MUN-" + outros_concelhos_distrito["id"].astype(str)

    return concelho_selecionado, outros_concelhos_distrito


def create_project(outros_distritos, outros_concelhos_distrito, concelho_selecionado):
    """Cria o projeto final concatenando todas as camadas."""
    return pd.concat([outros_distritos, outros_concelhos_distrito, concelho_selecionado],
                    ignore_index=True)


def save_project(projeto_gdf, concelho_nome):
    """Salva o projeto em arquivo GeoJSON."""
    output_filename = f"{concelho_nome.replace(' ', '_')}_AOI.geojson"
    try:
        projeto_gdf.to_file(output_filename, driver='GeoJSON')
        print(f"Projeto salvo com sucesso em {output_filename}")
    except Exception as e:
        raise IOError(f"Erro ao salvar projeto: {e}")


def main():
    """Função principal do programa."""
    try:
        distrito, concelho = get_user_input()

        distritos = load_geodata("distritos_portugal_SIMP.geojson")
        concelhos = load_geodata("concelhos_portugal_SIMP.geojson")

        outros_distritos = filter_distrito(distritos, distrito)
        concelho_selecionado, outros_concelhos_distrito = process_concelhos(concelhos, concelho, distrito)

        projeto = create_project(outros_distritos, outros_concelhos_distrito, concelho_selecionado)
        save_project(projeto, concelho)

    except (FileNotFoundError, ValueError, IOError) as e:
        print(f"Erro: {e}")
    except KeyboardInterrupt:
        print("\nOperação cancelada pelo usuário.")
    except Exception as e:
        print(f"Erro inesperado: {e}")


if __name__ == "__main__":
    main()

Informe o nome do distrito (ex: Évora, Lisboa, Porto):  Lisboa
Informe o nome do concelho (ex: Évora, Sintra, Matosinhos):  Lisboa


Projeto salvo com sucesso em Lisboa_AOI.geojson


In [2]:
teste = gpd.read_file("Lisboa_AOI.geojson")

In [3]:
teste

,id,name,poly_type,distrito,geometry
0,PRO-1001,Évora,periphery,None,"POLYGON ((-7.20378 38.75151, -7.20409 38.75184..."
1,PRO-1002,Aveiro,periphery,None,"MULTIPOLYGON (((-8.68208 40.69458, -8.68347 40..."
2,PRO-1003,Açores,periphery,None,"MULTIPOLYGON (((-25.0891 37.01562, -25.08908 3..."
3,PRO-1004,Beja,periphery,None,"MULTIPOLYGON (((-8.79264 37.71875, -8.79347 37..."
4,PRO-1005,Braga,periphery,None,"POLYGON ((-7.97415 41.37519, -7.97384 41.37542..."
5,PRO-1006,Bragança,periphery,None,"POLYGON ((-6.93135 41.0294, -6.93065 41.02971,..."
6,PRO-1007,Castelo Branco,periphery,None,"POLYGON ((-6.95142 40.25756, -6.95341 40.25806..."
7,PRO-1008,Coimbra,periphery,None,"POLYGON ((-8.35621 40.36723, -8.35617 40.36713..."
8,PRO-1009,Faro,periphery,None,"MULTIPOLYGON (((-7.88042 36.97347, -7.88153 36..."
9,PRO-1010,Guarda,periphery,None,"MULTIPOLYGON (((-7.40951 40.92181, -7.41465 40..."


In [4]:
cores = gpd.read_file("DEMO SCML.geojson")

In [6]:
cores = cores[["id", "name", "poly_type", "geometry"]]

In [8]:
cores = cores[cores["poly_type"] == "core"]

In [10]:
cores.to_file("DEMO SCML.geojson")

In [11]:
cores

,id,name,poly_type,geometry
0,998,SCML Lg Trindade Coelho,core,"POLYGON ((-9.14249 38.71273, -9.14375 38.71273..."
1,999,SCML Av. Liberdade,core,"POLYGON ((-9.14488 38.72063, -9.14666 38.72063..."
